# COSMOS: 多机器人编队导航演示

**COSMOS** (COordinated Safety On Manifold for multi-agent Systems) + RMPflow + MAPPO

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ustbmicl-ros2epucksRL/safe-rl-manifold-suite/blob/master/formation_nav/COSMOS_Demo.ipynb)

---

## 1. 环境配置

In [ ]:
# @title 版本与源码配置 { run: "auto" }
from datetime import datetime

# Notebook 版本 — 用于检测仓库中是否有更新版本
NOTEBOOK_VERSION = "2026-02-23-v2"

# ----- 源码配置 -----
GITHUB_ORG = "ustbmicl-ros2epucksRL"  # @param {type: "string"}
GITHUB_REPO = "safe-rl-manifold-suite"  # @param {type: "string"}
BRANCH = "master"  # @param {type: "string"}

# ----- 实验配置 -----
EXPERIMENT_NAME = "cosmos_exp"  # @param {type: "string"}
RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")  # 自动生成运行ID

print(f"Notebook 版本: {NOTEBOOK_VERSION}")
print(f"源码仓库:      {GITHUB_ORG}/{GITHUB_REPO}")
print(f"分支:          {BRANCH}")
print(f"实验名称:      {EXPERIMENT_NAME}")
print(f"运行ID:        {RUN_ID}")

In [ ]:
# 安装依赖
!pip install gymnasium wandb -q

# 验证环境
import torch
import numpy as np

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 配置 Keys (GitHub & WandB)

In [ ]:
# @title 配置 API Keys { run: "auto" }
from google.colab import userdata
import os

# 方式1: 从 Colab Secrets 读取 (推荐)
# 在 Colab 左侧 🔑 图标添加 secrets: GITHUB_TOKEN, WANDB_API_KEY
try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("✓ GitHub Token 已从 Secrets 加载")
except:
    GITHUB_TOKEN = None
    print("⚠ GitHub Token 未设置 (如果仓库是公开的则不需要)")

try:
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    print("✓ WandB API Key 已从 Secrets 加载")
except:
    WANDB_API_KEY = None
    print("⚠ WandB API Key 未设置 (将使用离线模式)")

In [ ]:
# 方式2: 手动输入 (如果 Secrets 未设置)
from getpass import getpass

if GITHUB_TOKEN is None:
    print("请输入 GitHub Token (留空则尝试公开克隆):")
    GITHUB_TOKEN = getpass("GitHub Token: ") or None

if WANDB_API_KEY is None:
    print("请输入 WandB API Key (留空则使用离线模式):")
    WANDB_API_KEY = getpass("WandB API Key: ") or None
    if WANDB_API_KEY:
        os.environ['WANDB_API_KEY'] = WANDB_API_KEY

## 3. 克隆代码库

In [ ]:
import os

# 先切换到 /content 目录（确保目录状态正常）
%cd /content

# 使用配置的仓库信息
REPO_URL = f"https://github.com/{GITHUB_ORG}/{GITHUB_REPO}.git"
REPO_PATH = f"/content/{GITHUB_REPO}"

# 检查是否已克隆
if os.path.exists(REPO_PATH):
    print(f"仓库已存在，更新代码...")
    %cd {REPO_PATH}
    !git fetch origin
    !git checkout {BRANCH}
    !git pull origin {BRANCH}
else:
    # 克隆仓库 (跳过 LFS 文件，参考文献 PDF 不影响代码运行)
    %env GIT_LFS_SKIP_SMUDGE=1
    if GITHUB_TOKEN:
        # 使用 Token 克隆私有仓库
        clone_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_ORG}/{GITHUB_REPO}.git"
        !git clone -b {BRANCH} {clone_url}
    else:
        # 尝试公开克隆
        !git clone -b {BRANCH} {REPO_URL}
    
    %cd {REPO_PATH}

print(f"\n当前目录: {os.getcwd()}")
print(f"分支:     {BRANCH}")
!git log --oneline -3

## 4. 导入模块

In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation
import time

from formation_nav.config import Config
from formation_nav.env.formation_env import FormationNavEnv
from formation_nav.env.formations import FormationTopology
from formation_nav.safety import COSMOS, COSMOSMode
from formation_nav.algo.mappo import MAPPO
from formation_nav.algo.buffer import RolloutBuffer

print("✓ 模块导入成功!")

## 5. 配置参数

In [ ]:
# @title 训练参数 { run: "auto" }

NUM_EPISODES = 100  # @param {type: "slider", min: 20, max: 500, step: 10}
NUM_AGENTS = 4  # @param [3, 4, 5, 6] {type: "raw"}
FORMATION = "square"  # @param ["square", "triangle", "circle", "line", "hexagon"]
SEED = 42  # @param {type: "integer"}

# ----- 日志与保存配置 -----
USE_WANDB = True  # @param {type: "boolean"}
SAVE_CHECKPOINT_EVERY = 20  # @param {type: "slider", min: 10, max: 100, step: 10}
RESUME_FROM_CHECKPOINT = ""  # @param {type: "string"}

print(f"训练轮数:     {NUM_EPISODES}")
print(f"智能体数量:   {NUM_AGENTS}")
print(f"编队形状:     {FORMATION}")
print(f"随机种子:     {SEED}")
print(f"使用 WandB:   {USE_WANDB}")
print(f"检查点间隔:   每 {SAVE_CHECKPOINT_EVERY} 轮")
print(f"断点续训:     {RESUME_FROM_CHECKPOINT or '从头开始'}")

In [ ]:
# @title 环境参数 { run: "auto" }

ARENA_SIZE = 5.0  # @param {type: "slider", min: 3.0, max: 10.0, step: 0.5}
NUM_OBSTACLES = 4  # @param {type: "slider", min: 0, max: 8, step: 1}
MAX_STEPS = 500  # @param {type: "slider", min: 100, max: 1000, step: 50}
DT = 0.02  # @param {type: "number"}
FORMATION_RADIUS = 1.0  # @param {type: "slider", min: 0.5, max: 2.0, step: 0.1}

print(f"场地大小:   {ARENA_SIZE} x {ARENA_SIZE}")
print(f"障碍物数量: {NUM_OBSTACLES}")
print(f"最大步数:   {MAX_STEPS}")
print(f"时间步长:   {DT}")
print(f"编队半径:   {FORMATION_RADIUS}")

In [ ]:
# @title 安全参数 (COSMOS) { run: "auto" }

SAFETY_RADIUS = 0.5  # @param {type: "slider", min: 0.2, max: 1.0, step: 0.1}
BOUNDARY_MARGIN = 0.5  # @param {type: "slider", min: 0.1, max: 1.0, step: 0.1}
COSMOS_MODE = "decentralized"  # @param ["centralized", "decentralized"]

print(f"安全半径:     {SAFETY_RADIUS}")
print(f"边界边距:     {BOUNDARY_MARGIN}")
print(f"COSMOS 模式:  {COSMOS_MODE}")

In [ ]:
# @title 算法参数 (MAPPO) { run: "auto" }

LEARNING_RATE = 3e-4  # @param {type: "number"}
PPO_EPOCHS = 10  # @param {type: "slider", min: 5, max: 20, step: 1}
NUM_MINI_BATCH = 4  # @param [2, 4, 8] {type: "raw"}
GAMMA = 0.99  # @param {type: "slider", min: 0.9, max: 0.999, step: 0.001}
GAE_LAMBDA = 0.95  # @param {type: "slider", min: 0.9, max: 1.0, step: 0.01}
CLIP_EPSILON = 0.2  # @param {type: "slider", min: 0.1, max: 0.3, step: 0.05}
ENTROPY_COEF = 0.01  # @param {type: "number"}

print(f"学习率:       {LEARNING_RATE}")
print(f"PPO 轮数:     {PPO_EPOCHS}")
print(f"Mini-batch:   {NUM_MINI_BATCH}")
print(f"折扣因子:     {GAMMA}")
print(f"GAE lambda:   {GAE_LAMBDA}")
print(f"Clip epsilon: {CLIP_EPSILON}")
print(f"熵系数:       {ENTROPY_COEF}")

In [ ]:
# 创建配置对象和输出目录
import json

np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 创建输出目录
OUTPUT_DIR = f"/content/outputs/{EXPERIMENT_NAME}/{RUN_ID}"
CHECKPOINT_DIR = f"{OUTPUT_DIR}/checkpoints"
RESULTS_DIR = f"{OUTPUT_DIR}/results"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

cfg = Config()

# 环境配置
cfg.env.num_agents = NUM_AGENTS
cfg.env.formation_shape = FORMATION
cfg.env.arena_size = ARENA_SIZE
cfg.env.num_obstacles = NUM_OBSTACLES
cfg.env.max_steps = MAX_STEPS
cfg.env.dt = DT
cfg.env.formation_radius = FORMATION_RADIUS

# 安全配置
cfg.safety.safety_radius = SAFETY_RADIUS
cfg.safety.boundary_margin = BOUNDARY_MARGIN

# 算法配置
cfg.algo.actor_lr = LEARNING_RATE
cfg.algo.critic_lr = LEARNING_RATE
cfg.algo.ppo_epochs = PPO_EPOCHS
cfg.algo.num_mini_batch = NUM_MINI_BATCH
cfg.algo.gamma = GAMMA
cfg.algo.gae_lambda = GAE_LAMBDA
cfg.algo.clip_param = CLIP_EPSILON
cfg.algo.entropy_coef = ENTROPY_COEF

# 训练配置
cfg.train.seed = SEED

# 保存配置到文件
config_dict = {
    "experiment_name": EXPERIMENT_NAME,
    "run_id": RUN_ID,
    "num_episodes": NUM_EPISODES,
    "num_agents": NUM_AGENTS,
    "formation": FORMATION,
    "seed": SEED,
    "arena_size": ARENA_SIZE,
    "safety_radius": SAFETY_RADIUS,
    "cosmos_mode": COSMOS_MODE,
    "learning_rate": LEARNING_RATE,
    "device": DEVICE,
}
with open(f"{OUTPUT_DIR}/config.json", "w") as f:
    json.dump(config_dict, f, indent=2)

print("=" * 50)
print("配置汇总")
print("=" * 50)
print(f"设备:         {DEVICE}")
print(f"智能体:       {cfg.env.num_agents} x {cfg.env.formation_shape}")
print(f"场地:         {cfg.env.arena_size} x {cfg.env.arena_size}")
print(f"安全半径:     {cfg.safety.safety_radius}")
print(f"COSMOS 模式:  {COSMOS_MODE}")
print(f"输出目录:     {OUTPUT_DIR}")
print("=" * 50)

## 6. 初始化 WandB (可选)

In [ ]:
# 初始化 WandB
import wandb

if USE_WANDB:
    wandb.init(
        project="cosmos-formation-nav",
        name=f"{EXPERIMENT_NAME}_{RUN_ID}",
        id=RUN_ID,
        resume="allow",  # 支持断点续训
        config=config_dict,
        tags=[FORMATION, f"n{NUM_AGENTS}", COSMOS_MODE],
    )
    # 保存代码
    wandb.run.log_code(".")
    print(f"✓ WandB 初始化成功")
    print(f"  项目: cosmos-formation-nav")
    print(f"  运行: {wandb.run.name}")
    print(f"  链接: {wandb.run.get_url()}")
else:
    print("跳过 WandB (USE_WANDB=False)")
    print("警告: 训练日志将不会保存到云端")

## 7. 创建环境和模型

In [ ]:
# 环境
env = FormationNavEnv(cfg.env, cfg.reward)
topology = FormationTopology(cfg.env.num_agents, "complete")

# COSMOS 模式
cosmos_mode = COSMOSMode.CENTRALIZED if COSMOS_MODE == "centralized" else COSMOSMode.DECENTRALIZED

# COSMOS 安全滤波器
cosmos = COSMOS(
    env_cfg=cfg.env,
    safety_cfg=cfg.safety,
    desired_distances=env.desired_distances,
    topology_edges=topology.edges(),
    obstacle_positions=env.obstacles,
    mode=cosmos_mode
)

# MAPPO
obs_dim = env.observation_space.shape[0]
share_obs_dim = env.share_observation_space.shape[0]

mappo = MAPPO(obs_dim, share_obs_dim, act_dim=2, cfg=cfg.algo, device=DEVICE)

# Buffer
buffer = RolloutBuffer(
    episode_length=cfg.env.max_steps,
    num_agents=cfg.env.num_agents,
    obs_dim=obs_dim,
    share_obs_dim=share_obs_dim,
    act_dim=2,
    gamma=cfg.algo.gamma,
    gae_lambda=cfg.algo.gae_lambda,
    device=DEVICE
)

# 断点续训: 加载检查点
start_episode = 0
metrics = {'episode': [], 'reward': [], 'cost': [], 'formation_error': [], 'min_dist': [], 'collisions': []}

if RESUME_FROM_CHECKPOINT and os.path.exists(RESUME_FROM_CHECKPOINT):
    print(f"从检查点恢复: {RESUME_FROM_CHECKPOINT}")
    checkpoint = torch.load(RESUME_FROM_CHECKPOINT, map_location=DEVICE)
    # 加载模型权重 (MAPPO 内部网络)
    mappo.actor.load_state_dict(checkpoint['actor'])
    mappo.critic.load_state_dict(checkpoint['critic'])
    if 'cost_critic' in checkpoint:
        mappo.cost_critic.load_state_dict(checkpoint['cost_critic'])
    start_episode = checkpoint.get('episode', 0)
    metrics = checkpoint.get('metrics', metrics)
    print(f"  已恢复到 Episode {start_episode}")
else:
    print("从头开始训练")

print(f"\n观测维度:    {obs_dim}")
print(f"共享观测:    {share_obs_dim}")
print(f"COSMOS 模式: {cosmos_mode.value}")
print(f"设备:        {DEVICE}")
print(f"起始轮数:    {start_episode}")
print("\n✓ 初始化完成!")

## 8. 训练

In [ ]:
# 训练循环 (支持断点续训和检查点保存)

def save_checkpoint(episode, mappo, metrics, path):
    """保存检查点 (与 MAPPO.save() 格式兼容)"""
    checkpoint = {
        'episode': episode,
        'actor': mappo.actor.state_dict(),
        'critic': mappo.critic.state_dict(),
        'cost_critic': mappo.cost_critic.state_dict(),
        'metrics': metrics,
        'config': config_dict,
    }
    torch.save(checkpoint, path)
    return path

print("开始训练...")
print("=" * 70)

start_time = time.time()
total_steps = 0
LOG_INTERVAL = max(1, NUM_EPISODES // 10)
best_reward = float('-inf')

for episode in range(start_episode, NUM_EPISODES):
    obs, share_obs, _ = env.reset(seed=cfg.train.seed + episode)
    cosmos.update_obstacles(env.obstacles)
    cosmos.reset(env.positions)
    buffer.set_first_obs(obs, share_obs)
    
    ep_reward, ep_cost, ep_form_err, ep_min_dist, ep_collisions = 0.0, 0.0, 0.0, float('inf'), 0
    
    for step in range(cfg.env.max_steps):
        alphas, log_probs = mappo.get_actions(obs)
        values = mappo.get_values(share_obs)
        safe_actions = cosmos.project(alphas, env.positions, env.velocities, dt=cfg.env.dt)
        next_obs, next_share_obs, rewards, costs, dones, infos, _ = env.step(safe_actions)
        
        masks = (~dones).astype(np.float32).reshape(-1, 1)
        buffer.insert(next_obs, next_share_obs, alphas, log_probs, values, rewards, costs, masks)
        
        obs, share_obs = next_obs, next_share_obs
        total_steps += cfg.env.num_agents
        
        ep_reward += rewards[0, 0]
        ep_cost += costs[0, 0]
        ep_form_err += infos[0]["formation_error"]
        ep_min_dist = min(ep_min_dist, infos[0]["min_inter_dist"])
        ep_collisions += infos[0]["collisions"]
        
        if dones.all():
            break
    
    # PPO 更新
    last_values = mappo.get_values(share_obs)
    buffer.compute_returns_and_advantages(last_values)
    update_info = mappo.update(buffer)
    buffer.after_update()
    
    # 记录指标
    avg_form_err = ep_form_err / (step + 1)
    metrics['episode'].append(episode + 1)
    metrics['reward'].append(ep_reward)
    metrics['cost'].append(ep_cost)
    metrics['formation_error'].append(avg_form_err)
    metrics['min_dist'].append(ep_min_dist)
    metrics['collisions'].append(ep_collisions)
    
    # WandB 日志
    if USE_WANDB:
        wandb.log({
            "train/reward": ep_reward,
            "train/cost": ep_cost,
            "train/formation_error": avg_form_err,
            "train/min_inter_dist": ep_min_dist,
            "train/collisions": ep_collisions,
            "train/episode_length": step + 1,
            "train/total_steps": total_steps,
            "episode": episode + 1,
        })
    
    # 保存检查点
    if (episode + 1) % SAVE_CHECKPOINT_EVERY == 0:
        ckpt_path = f"{CHECKPOINT_DIR}/checkpoint_ep{episode+1}.pt"
        save_checkpoint(episode + 1, mappo, metrics, ckpt_path)
        print(f"  [检查点已保存: {ckpt_path}]")
        
        # 上传到 WandB
        if USE_WANDB:
            artifact = wandb.Artifact(f"checkpoint_ep{episode+1}", type="model")
            artifact.add_file(ckpt_path)
            wandb.log_artifact(artifact)
    
    # 保存最佳模型
    if ep_reward > best_reward:
        best_reward = ep_reward
        best_path = f"{CHECKPOINT_DIR}/best_model.pt"
        save_checkpoint(episode + 1, mappo, metrics, best_path)
    
    # 打印进度
    if (episode + 1) % LOG_INTERVAL == 0:
        elapsed = time.time() - start_time
        fps = total_steps / elapsed
        print(f"Ep {episode+1:4d}/{NUM_EPISODES} | R={ep_reward:7.2f} | Cost={ep_cost:4.0f} | "
              f"FormErr={avg_form_err:.4f} | MinDist={ep_min_dist:.3f} | Coll={ep_collisions:2d} | FPS={fps:.0f}")

# 保存最终模型
final_path = f"{CHECKPOINT_DIR}/final_model.pt"
save_checkpoint(NUM_EPISODES, mappo, metrics, final_path)

elapsed = time.time() - start_time
print("=" * 70)
print(f"训练完成! 用时: {elapsed:.1f}s | 总碰撞: {sum(metrics['collisions'])}")
print(f"模型已保存: {final_path}")

## 9. 训练曲线

In [ ]:
# 训练曲线 (保存到文件)
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

axes[0, 0].plot(metrics['episode'], metrics['reward'], 'b-', alpha=0.7)
axes[0, 0].set_title('Episode Reward')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(metrics['episode'], metrics['cost'], 'r-', alpha=0.7)
axes[0, 1].set_title('Episode Cost')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(metrics['episode'], metrics['formation_error'], 'g-', alpha=0.7)
axes[0, 2].set_title('Formation Error')
axes[0, 2].set_xlabel('Episode')
axes[0, 2].grid(True, alpha=0.3)

axes[1, 0].plot(metrics['episode'], metrics['min_dist'], 'm-', alpha=0.7)
axes[1, 0].axhline(y=cfg.safety.safety_radius, color='r', linestyle='--', label='Safety Radius')
axes[1, 0].set_title('Min Inter-Agent Distance')
axes[1, 0].set_xlabel('Episode')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(metrics['episode'], metrics['collisions'], 'c-', alpha=0.7)
axes[1, 1].set_title('Collisions per Episode')
axes[1, 1].set_xlabel('Episode')
axes[1, 1].grid(True, alpha=0.3)

# Smoothed reward
window = min(20, len(metrics['episode']) // 5) or 1
if window > 1:
    smoothed = np.convolve(metrics['reward'], np.ones(window)/window, mode='valid')
    axes[1, 2].plot(metrics['episode'][window-1:], smoothed, 'b-', linewidth=2)
else:
    axes[1, 2].plot(metrics['episode'], metrics['reward'], 'b-', linewidth=2)
axes[1, 2].set_title(f'Smoothed Reward (window={window})')
axes[1, 2].set_xlabel('Episode')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()

# 保存到文件
curves_path = f"{RESULTS_DIR}/training_curves.png"
plt.savefig(curves_path, dpi=150, bbox_inches='tight')
print(f"训练曲线已保存: {curves_path}")

plt.show()

# 上传到 WandB
if USE_WANDB:
    wandb.log({"charts/training_curves": wandb.Image(fig)})

## 10. 评估与可视化

In [ ]:
# 模型验证 (多次评估并保存结果)

NUM_EVAL_EPISODES = 10  # 评估轮数
eval_results = []

print(f"开始验证 ({NUM_EVAL_EPISODES} 轮)...")
print("-" * 50)

for eval_ep in range(NUM_EVAL_EPISODES):
    obs, share_obs, _ = env.reset(seed=SEED + 1000 + eval_ep)
    cosmos.update_obstacles(env.obstacles)
    cosmos.reset(env.positions)
    
    trajectory = [env.positions.copy()]
    ep_reward, ep_collisions, ep_form_err = 0.0, 0, 0.0
    
    for step in range(cfg.env.max_steps):
        alphas, _ = mappo.get_actions(obs, deterministic=True)
        safe_actions = cosmos.project(alphas, env.positions, env.velocities, dt=cfg.env.dt)
        obs, share_obs, rewards, _, dones, infos, _ = env.step(safe_actions)
        trajectory.append(env.positions.copy())
        
        ep_reward += rewards[0, 0]
        ep_form_err += infos[0]["formation_error"]
        ep_collisions += infos[0]["collisions"]
        
        if dones.all():
            break
    
    # Convert numpy types to native Python types for JSON serialization
    result = {
        "episode": eval_ep + 1,
        "reward": float(ep_reward),
        "collisions": int(ep_collisions),
        "formation_error": float(ep_form_err / (step + 1)),
        "steps": step + 1,
        "min_dist": float(infos[0]["min_inter_dist"]),
    }
    eval_results.append(result)
    print(f"Eval {eval_ep+1:2d}: R={ep_reward:7.2f} | Coll={ep_collisions} | FormErr={result['formation_error']:.4f}")

# 保存最后一轮的轨迹用于可视化
trajectory = np.array(trajectory)
obstacles_pos = env.obstacles.copy()
goal_pos = env.goal.copy()

# 计算汇总统计 (convert to Python float for JSON)
avg_reward = float(np.mean([r['reward'] for r in eval_results]))
avg_collisions = float(np.mean([r['collisions'] for r in eval_results]))
avg_form_err = float(np.mean([r['formation_error'] for r in eval_results]))
total_collisions = sum([r['collisions'] for r in eval_results])

print("-" * 50)
print(f"平均奖励:     {avg_reward:.2f}")
print(f"平均碰撞:     {avg_collisions:.2f}")
print(f"总碰撞数:     {total_collisions}")
print(f"平均编队误差: {avg_form_err:.4f}")

# 保存验证结果
eval_results_path = f"{RESULTS_DIR}/eval_results.json"
with open(eval_results_path, 'w') as f:
    json.dump({
        "results": eval_results,
        "summary": {
            "avg_reward": avg_reward,
            "avg_collisions": avg_collisions,
            "total_collisions": total_collisions,
            "avg_formation_error": avg_form_err,
            "num_episodes": NUM_EVAL_EPISODES,
        }
    }, f, indent=2)
print(f"\n验证结果已保存: {eval_results_path}")

# 上传到 WandB
if USE_WANDB:
    wandb.log({
        "eval/avg_reward": avg_reward,
        "eval/avg_collisions": avg_collisions,
        "eval/total_collisions": total_collisions,
        "eval/avg_formation_error": avg_form_err,
    })
    
    # 上传结果文件
    artifact = wandb.Artifact("eval_results", type="results")
    artifact.add_file(eval_results_path)
    wandb.log_artifact(artifact)

In [ ]:
# 绘制轨迹并保存
fig, ax = plt.subplots(figsize=(10, 10))
colors = plt.cm.tab10(np.linspace(0, 1, cfg.env.num_agents))

# 场地边界
rect = patches.Rectangle((-cfg.env.arena_size, -cfg.env.arena_size), 
                          2*cfg.env.arena_size, 2*cfg.env.arena_size,
                          linewidth=2, edgecolor='black', facecolor='none', linestyle='--')
ax.add_patch(rect)

# 障碍物
for obs in obstacles_pos:
    circle = patches.Circle((obs[0], obs[1]), obs[2], facecolor='gray', edgecolor='black', alpha=0.5)
    ax.add_patch(circle)

# 目标
ax.plot(goal_pos[0], goal_pos[1], 'r*', markersize=25, label='Goal')

# 轨迹
for i in range(cfg.env.num_agents):
    traj = trajectory[:, i, :]
    ax.plot(traj[:, 0], traj[:, 1], '-', color=colors[i], alpha=0.7, linewidth=2)
    ax.plot(traj[0, 0], traj[0, 1], 'o', color=colors[i], markersize=12, label=f'Agent {i}')
    ax.plot(traj[-1, 0], traj[-1, 1], 's', color=colors[i], markersize=12)

# 最终编队
final_pos = trajectory[-1]
for (i, j) in topology.edges():
    ax.plot([final_pos[i, 0], final_pos[j, 0]], [final_pos[i, 1], final_pos[j, 1]], 'k--', alpha=0.5)

ax.set_xlim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
ax.set_ylim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
ax.set_aspect('equal')
ax.set_title('COSMOS + RMPflow + MAPPO Formation Navigation', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# 保存到文件
traj_path = f"{RESULTS_DIR}/trajectory.png"
plt.savefig(traj_path, dpi=150, bbox_inches='tight')
print(f"轨迹图已保存: {traj_path}")

plt.show()

# 上传到 WandB
if USE_WANDB:
    wandb.log({"charts/trajectory": wandb.Image(fig)})

## 11. 动画

In [ ]:
%%capture
# 创建动画
fig, ax = plt.subplots(figsize=(8, 8))
colors = plt.cm.tab10(np.linspace(0, 1, cfg.env.num_agents))
T = len(trajectory)

def init():
    ax.clear()
    rect = patches.Rectangle((-cfg.env.arena_size, -cfg.env.arena_size), 
                              2*cfg.env.arena_size, 2*cfg.env.arena_size,
                              linewidth=2, edgecolor='black', facecolor='none', linestyle='--')
    ax.add_patch(rect)
    for obs in obstacles_pos:
        circle = patches.Circle((obs[0], obs[1]), obs[2], facecolor='gray', edgecolor='black', alpha=0.5)
        ax.add_patch(circle)
    ax.plot(goal_pos[0], goal_pos[1], 'r*', markersize=20)
    ax.set_xlim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
    ax.set_ylim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
    ax.set_aspect('equal')
    return []

def update(frame):
    ax.clear()
    init()
    pos = trajectory[frame]
    trail_start = max(0, frame - 30)
    for i in range(cfg.env.num_agents):
        trail = trajectory[trail_start:frame+1, i, :]
        ax.plot(trail[:, 0], trail[:, 1], '-', color=colors[i], alpha=0.5, linewidth=2)
        ax.plot(pos[i, 0], pos[i, 1], 'o', color=colors[i], markersize=12)
    for (i, j) in topology.edges():
        ax.plot([pos[i, 0], pos[j, 0]], [pos[i, 1], pos[j, 1]], 'k-', alpha=0.3)
    ax.set_title(f'Step {frame}/{T-1}')
    return []

frames = list(range(0, T, max(1, T // 100)))
anim = FuncAnimation(fig, update, init_func=init, frames=frames, blit=False, interval=50)

In [ ]:
# 显示动画
from IPython.display import HTML
HTML(anim.to_jshtml())

## 12. 安全统计

In [ ]:
# 最终统计与 WandB 完成
total_train_collisions = sum(metrics['collisions'])
safe_episodes = sum(1 for c in metrics['collisions'] if c == 0)

print("=" * 60)
print("COSMOS 训练与验证汇总")
print("=" * 60)
print(f"\n【训练统计】")
print(f"  总轮数:         {NUM_EPISODES}")
print(f"  总碰撞次数:     {total_train_collisions}")
print(f"  零碰撞轮数:     {safe_episodes}/{NUM_EPISODES} ({100*safe_episodes/NUM_EPISODES:.1f}%)")
print(f"  最终编队误差:   {metrics['formation_error'][-1]:.4f}")
print(f"  最终最小距离:   {metrics['min_dist'][-1]:.3f}")

print(f"\n【验证统计】")
print(f"  评估轮数:       {NUM_EVAL_EPISODES}")
print(f"  平均奖励:       {avg_reward:.2f}")
print(f"  平均碰撞:       {avg_collisions:.2f}")
print(f"  平均编队误差:   {avg_form_err:.4f}")

print(f"\n【保存文件】")
print(f"  输出目录:       {OUTPUT_DIR}")
print(f"  检查点目录:     {CHECKPOINT_DIR}")
print(f"  结果目录:       {RESULTS_DIR}")

# 列出所有保存的文件
print(f"\n  已保存的模型:")
for f in os.listdir(CHECKPOINT_DIR):
    print(f"    - {f}")

print("=" * 60)

# 完成 WandB 运行
if USE_WANDB:
    # 记录最终汇总
    wandb.run.summary["train/total_collisions"] = total_train_collisions
    wandb.run.summary["train/safe_episode_rate"] = safe_episodes / NUM_EPISODES
    wandb.run.summary["eval/avg_reward"] = avg_reward
    wandb.run.summary["eval/total_collisions"] = total_collisions
    
    # 上传所有结果文件
    artifact = wandb.Artifact(f"results_{RUN_ID}", type="results")
    artifact.add_dir(RESULTS_DIR)
    wandb.log_artifact(artifact)
    
    # 上传最终模型
    model_artifact = wandb.Artifact(f"model_{RUN_ID}", type="model")
    model_artifact.add_dir(CHECKPOINT_DIR)
    wandb.log_artifact(model_artifact)
    
    wandb.finish()
    print("\n✓ WandB 运行已完成，所有数据已上传")